# **Latihan Number, Date Format, & SubQuery**
**Putu Bayuwestra** w/ DQ Lab

# <b><span style='color:#0B2F9F'>Pengantar</span></b>

Dalam pengelolaan data, kemampuan untuk menampilkan informasi dengan format yang tepat serta menggali data secara mendalam menjadi hal yang sangat penting. Format angka dan tanggal membantu memastikan data tersaji secara konsisten, mudah dibaca, dan sesuai konteks, sehingga hasil analisis lebih akurat dan informatif. Sementara itu, penggunaan subquery memungkinkan pengambilan data secara lebih fleksibel dan efisien, dengan cara memecah proses pencarian menjadi langkah-langkah yang lebih terarah. Dengan memahami kedua konsep ini, seseorang dapat mengolah data dengan lebih rapi, melakukan analisis yang lebih mendalam, serta menghasilkan informasi yang lebih bermakna dan dapat diandalkan.


# <b><span style='color:#0B2F9F'>Set up</span></b>

Dibutuhkan proses autentikasi dari Google Colab ke Google Big Query. Ikuti langkah berikut https://drive.google.com/file/d/1gW8alZ_PrvcrsieqWCHOR4ssLI_25BRc/view untuk detail step-by-step nya

In [1]:
# Import library yang dibutuhkan
from google.colab import auth, data_table
from google.cloud import bigquery
from pandas_gbq import to_gbq

# Proses autentikasi akun
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# Buat BigQuery client
project_id = 'bay-data-475711'
client = bigquery.Client(project = project_id)

Sebagai Data Engineer, Antara (Jr. Data Engineer di DQPizza) ditugaskan untuk membuat data mart untuk kebutuhan detail transaksi. Dengan memanfaatkan relasi antar table, Antara mengeksekusi suatu perintah  untuk membuat table f_detail_transaksi (sebuah tabel fact yang berisi data transaksi secara rinci, mencakup ukuran-ukuran kuantitatif seperti jumlah, harga, total nilai transaksi, dan lain sebagainya) hingga terbentuklah tabel f_detail_transaksi.

<br>
<code>SELECT * FROM dqlab-9876543.dq_pizza.f_detail_transaksi</code>
<br><br>
<img src="https://raw.githubusercontent.com/bachtiyarma/Material/main/Image/Materi-SQL/SQL%20-%20f_detail_transaksi.png" width="30%">

Data dibawah (f_detail_transaksi) akan digunakan pada modul ini

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  *
FROM `dqlab-9876543.dq_pizza.f_detail_transaksi`

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None


# <b>A. <span style='color:#0B2F9F'><code>Data Type</code></span></b>

Data type dalam BigQuery adalah cara untuk mendefinisikan jenis data yang akan disimpan dalam tabel. Setiap kolom dalam tabel memiliki tipe data tertentu, yang menentukan apa yang dapat disimpan di kolom tersebut dan bagaimana data tersebut dapat diproses. Memahami tipe data ini penting untuk desain skema yang efisien dan untuk memastikan bahwa kueri dapat dijalankan dengan optimal.

<br>
<img src="https://hackolade.com/help/lib/BigQuery%20data%20types%20table.png" width="40%">

docs : <i><a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types">https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types</a></i>

# <b>B. <span style='color:#0B2F9F'><code>Operasi Tipe Data Numerik</code></span></b>

Dalam SQL, tipe data numerik digunakan untuk menyimpan nilai angka dan dapat dikelompokkan menjadi beberapa jenis sesuai peruntukannya (INT, FLOAT, NUMERIC, dll). Selain itu pada tipe data Numerik juga dapat dilakukan operasi matematis diantaranya :
<ol>
    <li>Operasi Aritmatika</li>
    <li>Fungsi Matematis</li>
    <li>Agregasi</li>
</ol>

## <b>B.1. <span style='color:#0B2F9F'><code>Operasi Aritmatika</code></span></b>
SQL mendukung operasi dasar aritmatika seperti penjumlahan, pengurangan, perkalian, pembagian dan modulo.

a = 10, b = 20

<img src="https://mazamanbd.wordpress.com/wp-content/uploads/2014/09/dd.jpg"><br><br>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL%20-%20OperasiAritmatika.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/operators">https://cloud.google.com/bigquery/docs/reference/standard-sql/operators</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel f_detail_transaksi, tampilkan semua kolom dan tambahkan satu kolom 'total_price' yang merupakan hasil perkalian antara harga dengan kuantitas (terbeli)! Urutkan berdasarkan kuantitas paling besar hingga ke terkecil </b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

/*
Harga Total diperoleh dari perkalian Harga per Item dengan Kuantitas
*/

SELECT *,
(price*quantity) AS total_price
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
ORDER BY quantity DESC;

Executing query with job ID: 1cea4fe5-892a-4f99-b1d0-9cf71be4b957
Query executing: 1.27s
Job ID 1cea4fe5-892a-4f99-b1d0-9cf71be4b957 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN


#### <i><b><span style='color:#55679C'>Quest</span> : Selain total price, tambahkan juga total profit yang didapat tiap transaksi! (Lanjutkan hasil query quest diatas) </b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price,total_profit
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0,62532.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0,62532.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0,196344.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0,108621.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0,108621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0,22356.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN,NaN


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT *,
  (price*quantity) AS total_price,
  ((price-production_cost)*quantity) AS total_profit
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
ORDER BY quantity DESC;

Executing query with job ID: 177fd8a0-a35e-4789-ba92-9636e4f2ea18
Query executing: 1.39s
Job ID 177fd8a0-a35e-4789-ba92-9636e4f2ea18 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,total_price,total_profit
0,TRX005264,TRXD011977,CUST004837,EMP000286,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,03/30/2024,13:24:35,13:35:43,0,None,388800.0,62532.0
1,TRX008286,TRXD018876,CUST004105,EMP000437,THE BIG MEAT PIZZA,Classic,S,97200.0,81567.0,4,05/19/2024,12:42:46,13:11:40,0,None,388800.0,62532.0
2,TRX015674,TRXD035497,CUST006814,EMP000302,THE CALIFORNIA CHICKEN PIZZA,CHICKEN,L,168075.0,118989.0,4,09/19/2024,14:16:35,14:40:53,0,None,672300.0,196344.0
3,TRX018845,TRXD042847,CUST007150,EMP000756,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,11/18/2024,12:25:12,12:50:38,0,None,407025.0,108621.0
4,TRX000144,TRXD000342,CUST006964,EMP000231,THE BARBECUE CHICKEN PIZZA,CHICKEN,M,135675.0,99468.0,3,01/03/2024,13:44:28,14:06:42,0,None,407025.0,108621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX018442,TRXD041913,CUST007150,EMP000437,THE MEDITERRANEAN PIZZA,VEGGIE,S,97200.0,74844.0,1,11/10/2024,19:49:13,20:02:01,0,None,97200.0,22356.0
48619,TRX006536,None,CUST001704,EMP000665,None,None,None,NaN,NaN,<NA>,04/20/2024,16:04:24,16:26:08,0,None,NaN,NaN
48620,TRX006537,None,CUST006034,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,16:39:18,17:02:44,0,None,NaN,NaN
48621,TRX006505,None,CUST001704,EMP000560,None,None,None,NaN,NaN,<NA>,04/20/2024,11:59:36,12:11:41,0,None,NaN,NaN


#### <i><b><span style='color:#55679C'>Quest</span> : Bagaimana dengan customer paling loyal? Paling loyal diukur dari banyak profit restaurant yang dihasilkan. Tampilkan customer_id, total_frequency (berapa kali pembelian dalam setahun) dan total_profit yang diperoleh perusahaan, kelompokkan tiap customer_id! Urutkan berdasarkan customer_id dengan profit paling tinggi ke paling rendah! </b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,customer_id,total_frequency,total_profit
0,CUST006333,741,47622169.0
1,CUST009381,702,44017426.0
2,CUST007656,614,37358334.0
3,CUST003549,580,36914292.0
4,CUST006964,521,32351562.0
...,...,...,...
115,CUST008078,21,1070496.0
116,CUST005536,24,987714.0
117,CUST006892,12,858681.0
118,CUST003871,13,685908.0


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

/*
Total Profit diperoleh dari Selisih Harga Jual dengan Harga Produksi dikali dengan Kuantitas Item
*/

SELECT
  customer_id,
  COUNT(DISTINCT(order_id)) AS total_frequency,
  SUM((price-production_cost)*quantity) AS total_profit
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
GROUP BY customer_id
ORDER BY total_profit DESC;

Executing query with job ID: 39598a40-281f-4b51-869f-a5a2cd47a813
Query executing: 0.36s
Job ID 39598a40-281f-4b51-869f-a5a2cd47a813 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_frequency,total_profit
0,CUST006333,741,47622169.0
1,CUST009381,702,44017426.0
2,CUST007656,614,37358334.0
3,CUST003549,580,36914292.0
4,CUST006964,521,32351562.0
...,...,...,...
115,CUST008078,21,1070496.0
116,CUST005536,24,987714.0
117,CUST006892,12,858681.0
118,CUST003871,13,685908.0


## <b>B.2. <span style='color:#0B2F9F'><code>Fungsi Matematis</code></span></b>
SQL juga mendukung fungsi matematis dasar yang disajikan pada tabel berikut :

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>ABS()</code></td>
            <td>Mengambil nilai absolut</td>
        </tr>
        <tr>
            <td><code>ROUND()</code></td>
            <td>Membulatkan angka</td>
        </tr>
        <tr>
            <td><code>CEIL()</code></td>
            <td>Membulatkan ke atas</td>
        </tr>
        <tr>
            <td><code>FLOOR()</code></td>
            <td>Membulatkan ke bawah</td>
        </tr>
        <tr>
            <td><code>POW()</code></td>
            <td>Menghitung pangkat</td>
        </tr>
    </tbody>
</table>

Untuk dapat menerapkannya pada sebuah tabel gunakan contoh format berikut

<img src="https://raw.githubusercontent.com/bachtiyarma/Material/refs/heads/main/Image/Materi-SQL/SQL-FungsiMatematis.png" width="50%">

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/mathematical_functions</a></i>

#### <i><b><span style='color:#55679C'>Quest</span> : Sendja (Data Analyst di DQPizza) baru menyadari bahwa profit yang ia hitung pada quest sebelumnya merupakan profit sebelum pajak. Asumsikan pajak yang perlu dikeluarkan adalah 11% tiap kali transaksi. Hitung profit_before_tax, profit_after_tax dan tax_paid tiap order_id dan customer_id! Lakukan perhitungan untuk setiap transaksi yang valid (order_details_id tidak kosong)!</b></i>

Note : Bulatkan profit after tax sebanyak 2 angka dibelakang koma agar lebih rapi

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,order_id,customer_id,profit_before_tax,profit_after_tax,tax_paid
0,TRX018845,CUST007150,747954.0,665679.06,82274.94
1,TRX010760,CUST001704,684855.0,609520.95,75334.05
2,TRX004482,CUST003570,518481.0,461448.09,57032.91
3,TRX006169,CUST007656,494586.0,440181.54,54404.46
4,TRX009331,CUST004537,483813.0,430593.57,53219.43
...,...,...,...,...,...
21341,TRX006864,CUST008004,10854.0,9660.06,1193.94
21342,TRX014887,CUST002604,10854.0,9660.06,1193.94
21343,TRX015013,CUST003570,10854.0,9660.06,1193.94
21344,TRX013817,CUST004547,10854.0,9660.06,1193.94


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

/*
Profit Setelah Pajak adalah Total Profit yang dikurangi dengan Keuntungan Sebelum Pajak dikalikan dengan Pajak
*/

SELECT
  order_id,
  customer_id,
  SUM((price-production_cost)*quantity) AS profit_before_tax,
  ROUND(SUM(((price-production_cost)*quantity) * (1-(11/100))),2) AS profit_after_tax,
  SUM(((price-production_cost)*quantity) * (11/100)) AS tax_paid
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
WHERE order_details_id IS NOT NULL
GROUP BY
  1, 2
ORDER BY profit_before_tax DESC;

Executing query with job ID: a0061c08-12ae-48bd-a15a-ddcf147b1f5b
Query executing: 7.56s
Job ID a0061c08-12ae-48bd-a15a-ddcf147b1f5b successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,customer_id,profit_before_tax,profit_after_tax,tax_paid
0,TRX018845,CUST007150,747954.0,665679.06,82274.94
1,TRX010760,CUST001704,684855.0,609520.95,75334.05
2,TRX004482,CUST003570,518481.0,461448.09,57032.91
3,TRX006169,CUST007656,494586.0,440181.54,54404.46
4,TRX009331,CUST004537,483813.0,430593.57,53219.43
...,...,...,...,...,...
21341,TRX006864,CUST008004,10854.0,9660.06,1193.94
21342,TRX014887,CUST002604,10854.0,9660.06,1193.94
21343,TRX015013,CUST003570,10854.0,9660.06,1193.94
21344,TRX013817,CUST004547,10854.0,9660.06,1193.94


# <b>C. <span style='color:#0B2F9F'><code>Operasi Tipe Data Date</code></span></b>

Dalam SQL, tipe data date digunakan untuk menyimpan atau mengoperasikan data tanggal. Tipe data tanggal dapat dilakukan operasi berikut, diantaranya :
<ol>
    <li>Menampilkan Tanggal & Waktu saat ini</li>
    <li>Menambahkan atau Mengurangi Tanggal</li>
    <li>Mendapatkan Bagian Tanggal</li>
    <li>Menghitung Selisih Tanggal</li>
</ol>

<br><b>NOTE</b><br><br>
Hal-hal yang perlu diperhatikan dalam penggunaan operasi `date` adalah :
<ul>
    <li>Format tanggal standar SQL adalah 'YYYY-MM-DD' atau tahun - bulan - tanggal, contoh : 2024-10-14 menunjukan tanggal 14 October 2024</li>
    <li>Mengakses tanggal pada SQL perlu menggunakan tanda apit petik satu, contoh : '2024-10-14'</li>
    <li>Pastikan nilai atau kolom tanggal tipe datanya adalah DATE atau TIMESTAMP, jika tidak maka bisa dikonversi menjadi <code>DATE(tanggal)</code> atau <code>TIMESTAMP(tanggal)</code></li>
</ul>

<i>docs : <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions">https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions</a></i>

## <b>C.1. <span style='color:#0B2F9F'><code>Menampilkan Tanggal & Waktu Saat Ini</code></span></b>

Untuk menampilkan tanggal dan waktu saat ini dapat menggunakan fungsi berikut
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>CURRENT_DATE()</code></td>
            <td>Mendapatkan Tanggal Hari ini Saja</td>
        </tr>
        <tr>
            <td><code>CURRENT_TIMESTAMP()</code></td>
            <td>Mendapatkan Tanggal & Waktu Hari ini (UTC)</td>
        </tr>
    </tbody>
</table>

UTC atau (Universal Time Coordinated) adalah waktu dasar yang tidak terpengaruh oleh zona waktu dan <i>daylight saving time</i>. UTC digunakan sebagai acuan untuk menghitung waktu di berbagai zona waktu di seluruh dunia (standar Greenwich Mean Time (London)). Selengkapnya baca <a href="https://internasional.kompas.com/read/2021/03/04/192042570/sejarah-penetapan-zona-waktu-di-dunia-hingga-usulan-penghapusannya?page=all">disini</a>.

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan semua kolom pada tabel f_detail_transaksi dan tambahkan kolom data_collection_time sebagai kolom metadata waktu pengambilan data</b></i><br>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,data_collection_time
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None,2025-10-30 15:01:51.566696
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None,2025-10-30 15:01:51.566696
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None,2025-10-30 15:01:51.566696
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None,2025-10-30 15:01:51.566696
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None,2025-10-30 15:01:51.566696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None,2025-10-30 15:01:51.566696
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None,2025-10-30 15:01:51.566696
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None,2025-10-30 15:01:51.566696
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None,2025-10-30 15:01:51.566696


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT *,
  DATETIME(CURRENT_TIMESTAMP(),"Asia/Makassar") AS data_collection_time
FROM dqlab-9876543.dq_pizza.f_detail_transaksi;

Executing query with job ID: 25ceffd0-e5ab-4531-91dc-5ac98fc10893
Query executing: 1.45s
Job ID 25ceffd0-e5ab-4531-91dc-5ac98fc10893 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_id,order_details_id,customer_id,order_maker_id,pizza_name,category,size,price,production_cost,quantity,order_date,order_time,completion_time,is_complain,complain_detail,data_collection_time
0,TRX000734,TRXD001645,CUST003549,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,01/12/2024,20:00:30,20:27:49,0,None,2025-11-02 17:55:17.362850
1,TRX004103,TRXD009366,CUST006714,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,03/10/2024,19:10:04,19:39:33,0,None,2025-11-02 17:55:17.362850
2,TRX001914,TRXD004305,CUST008674,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,02/01/2024,19:24:03,19:36:58,0,None,2025-11-02 17:55:17.362850
3,TRX017344,TRXD039387,CUST004020,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,10/22/2024,13:20:05,13:39:42,0,None,2025-11-02 17:55:17.362850
4,TRX019817,TRXD045082,CUST003570,EMP000208,THE PEPPERONI PIZZA,CLASSIC,S,78975.0,62937.0,1,12/03/2024,16:40:44,17:07:19,0,None,2025-11-02 17:55:17.362850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48618,TRX016165,TRXD036640,CUST005956,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,09/29/2024,21:05:44,21:33:24,0,None,2025-11-02 17:55:17.362850
48619,TRX008475,TRXD019280,CUST004817,EMP000756,THE BRIE CARRE PIZZA,supreme,S,191565.0,170505.0,1,05/22/2024,16:56:23,17:20:01,0,None,2025-11-02 17:55:17.362850
48620,TRX001581,TRXD003564,CUST009381,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,01/27/2024,16:28:38,16:54:32,0,None,2025-11-02 17:55:17.362850
48621,TRX016753,TRXD037944,CUST004945,EMP000756,THE GREEK PIZZA,CLASSIC,XXL,291195.0,260738.0,1,10/10/2024,17:54:55,18:06:01,0,None,2025-11-02 17:55:17.362850


## <b>C.2. <span style='color:#0B2F9F'><code>Mengubah String menjadi Tanggal</code></span></b>

Adakalanya format tanggal yang disimpan pada database tidak disimpan dalam format date yang baku (<a href="https://www.iso.org/iso-8601-date-and-time-format.html">ISO 8601</a>) sehingga perlu penyesuaian. Gunakan : <br><br>
<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>PARSE_DATE(format, string_expression)</code></td>
            <td>Mengubah String menjadi Date</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE(format, date_expression)</code></td>
            <td>Mengubah Date menjadi String</td>
        </tr>
    </tbody>
</table><br>

Untuk detail format tanggal akses link berikut https://docs.cloud.google.com/bigquery/docs/reference/standard-sql/format-elements

#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan order_date dan order_date_iso pada tabel f_detail_transaksi. Dimana kolom order_date merupakan kolom dengan data asli dan order_date_iso merupakan kolom order_date yang diubah formatnya menjadi YYYY-MM-DD sesuai ISO 8601!</b></i><br>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

Executing query with job ID: 61b5b5dc-d646-461b-a540-4e21ed09862b
Query executing: 2.21s
Job ID 61b5b5dc-d646-461b-a540-4e21ed09862b successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_date,order_date_iso
0,01/12/2024,2024-01-12
1,03/10/2024,2024-03-10
2,02/01/2024,2024-02-01
3,10/22/2024,2024-10-22
4,12/03/2024,2024-12-03
...,...,...
48618,11/27/2024,2024-11-27
48619,11/27/2024,2024-11-27
48620,11/27/2024,2024-11-27
48621,11/27/2024,2024-11-27


**Latihan Query Mandiri :**

In [11]:
%%bigquery --project {project_id} --verbose

SELECT
  order_date,
  FORMAT_DATE('%Y-%m-%d', order_date) AS order_date_iso
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
ORDER BY order_date;

Executing query with job ID: b41af664-f1fe-45c5-a7f9-b3712600fe00
Query executing: 0.44s
Job ID b41af664-f1fe-45c5-a7f9-b3712600fe00 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,order_date,order_date_iso
0,2024-01-01,2024-01-01
1,2024-01-01,2024-01-01
2,2024-01-01,2024-01-01
3,2024-01-01,2024-01-01
4,2024-01-01,2024-01-01
...,...,...
48614,2024-12-31,2024-12-31
48615,2024-12-31,2024-12-31
48616,2024-12-31,2024-12-31
48617,2024-12-31,2024-12-31


## <b>C.3. <span style='color:#0B2F9F'><code>Menambahkan atau Mengurangi Tanggal</code></span></b>

<table border="1">
    <thead>
        <tr>
            <th>Fungsi</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>DATE_ADD(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Menambahkan Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_SUB(tanggal, INTERVAL jumlah_penambahan satuan_waktu)</code></td>
            <td>Mengurangi Tanggal pada Interval Tertentu</td>
        </tr>
        <tr>
            <td><code>DATE_DIFF(end_date, start_date, date_part)</code></td>
            <td>Menghitung selisih tanggal</td>
        </tr>
    </tbody>
</table><br>

Sebagai contoh misalkan hari ini tanggal 30 Oktober 2025. Ingin diperoleh tanggal tepat 3 bulan yang lalu maka dapat digunakan sintaks `DATE_SUB(DATE('2025-10-30'), INTERVAL 3 MONTH)` maka akan diperoleh tanggal 30 Juli 2025.



#### <i><b><span style='color:#55679C'>Quest</span> : Manajer ingin melakukan analisa demografi customer, pada tabel customers tampilkan customer_id, customer_name, gender dan age-nya per hari ini!</b></i><br>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,customer_id,customer_name,gender,age
0,CUST006458,Victoria Hutapea,F,42
1,CUST005844,Dewi Mayasari,F,39
2,CUST001212,Ani Habibi,F,44
3,CUST003654,Oliva Pratiwi,F,35
4,CUST003857,Kiandra Padmasari,F,25
...,...,...,...,...
132,CUST005303,Dwi Saefullah,M,28
133,CUST000968,Putu Wijaya,M,28
134,CUST008990,Rangga Nababan,M,18
135,CUST009522,Cakrawangsa Hutagalung,M,36


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  customer_id,
  customer_name,
  gender,
  DATE_DIFF(CURRENT_DATE(),birth_date,YEAR) AS age
FROM dqlab-9876543.dq_pizza.customers;

Executing query with job ID: 73217702-a95e-498b-b1fd-12f7e4aeac32
Query executing: 0.53s
Job ID 73217702-a95e-498b-b1fd-12f7e4aeac32 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_name,gender,age
0,CUST006458,Victoria Hutapea,F,42
1,CUST005844,Dewi Mayasari,F,39
2,CUST001212,Ani Habibi,F,44
3,CUST003654,Oliva Pratiwi,F,35
4,CUST003857,Kiandra Padmasari,F,25
...,...,...,...,...
132,CUST005303,Dwi Saefullah,M,28
133,CUST000968,Putu Wijaya,M,28
134,CUST008990,Rangga Nababan,M,18
135,CUST009522,Cakrawangsa Hutagalung,M,36


## <b>C.5. <span style='color:#0B2F9F'><code>Mendapatkan Bagian pada Tanggal</code></span></b>

Mengambil bagian tertentu dari tipe data tanggal, waktu, atau timestamp terkadang sangat perlu dilakukan untuk analisa data. Jika ingin didapatkan elemen tertentu, seperti tahun, bulan, hari, atau jam, dari nilai tanggal atau waktu gunakan fungsi berikut<br><br>

<code>EXTRACT(bagian FROM tanggal)</code>
<br><br>

Bagian yang bisa diambil pada tanggal diantaranya adalah
<table border="1">
    <thead>
        <tr>
            <th>Bagian</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>YEAR</code></td>
            <td>Mendapatkan Tahun pada Tanggal</td>
        </tr>
        <tr>
            <td><code>MONTH</code></td>
            <td>Mendapatkan Bulan pada Tanggal</td>
        </tr>
        <tr>
            <td><code>DAY</code></td>
            <td>Mendapatkan Hari (1-31) pada Tanggal</td>
        </tr>
        <tr>
            <td><code>WEEK</code></td>
            <td>Mendapatkan Minggu ke- pada Tanggal</td>
        </tr>
         <tr>
            <td><code>QUARTER</code></td>
            <td>Mendapatkan Kuartal pada Tanggal</td>
        </tr>
        <tr>
            <td><code>HOUR</code></td>
            <td>Mendapatkan Jam pada Waktu</td>
        </tr>
        <tr>
            <td><code>MINUTE</code></td>
            <td>Mendapatkan Menit pada Waktu</td>
        </tr>
        <tr>
            <td><code>SECOND</code></td>
            <td>Mendapatkan Detik pada Waktu</td>
        </tr>
    </tbody>
</table>
<br>

Namun jika menginginkan bagian yang lebih detail seperti Nama Bulan atau Nama Hari pada tanggal tertentu atau menampilkan format tanggal sesuai dengan keinginan dapat menggunakan sintaks `FORMAT_DATE(format_element, tanggal)`. <br><br>

<table border="1">
    <thead>
        <tr>
            <th>Sintaks</th>
            <th>Deskripsi</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td><code>FORMAT_DATE('%A', tanggal)</code></td>
            <td>Mendapatkan Nama Hari pada Tanggal Tertentu</td>
        </tr>
        <tr>
            <td><code>FORMAT_DATE('%B', tanggal)</code></td>
            <td>Mendapatkan Nama Bulan pada Tanggal Tertentu</td>
        </tr>
    </tbody>
</table>
<br>

Detail format_element dapat diakses dilaman <a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/format-elements#format_elements_date_time"> berikut</a>.





#### <i><b><span style='color:#55679C'>Quest</span> : Hitung total transaksi dan total pendapatan yang diperoleh tiap bulan pada tahun 2024 di DQPizza! </b></i><br> tabel : year, month, month_name, total_transaction, total_revenue

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,year,month,month_name,total_transaction,total_revenue
0,2024,1,January,1845,565325730.0
1,2024,2,February,1685,527792760.0
2,2024,3,March,1840,570216510.0
3,2024,4,April,1799,556243605.0
4,2024,5,May,1853,578362275.0
5,2024,6,June,1773,552664620.0
6,2024,7,July,1935,587718990.0
7,2024,8,August,1841,553053825.0
8,2024,9,September,1661,520073055.0
9,2024,10,October,1646,518623560.0


**Latihan Query Mandiri :**

In [ ]:
%%bigquery --project {project_id} --verbose

SELECT
  EXTRACT (YEAR FROM PARSE_DATE('%m/%d/%Y', order_date)) AS year,
  EXTRACT (MONTH FROM PARSE_DATE('%m/%d/%Y', order_date)) AS month,
  FORMAT_DATE ('%B',PARSE_DATE('%m/%d/%Y', order_date)) AS month_name,
  COUNT(order_id) AS total_transaction,
  SUM(price * quantity) AS total_revenue
FROM dqlab-9876543.dq_pizza.f_detail_transaksi
GROUP BY
  1, 2, 3
ORDER BY
  1, 2;

Executing query with job ID: 1b445040-0d1e-49b5-bffc-d6e17b3119f7
Query executing: 0.53s
Job ID 1b445040-0d1e-49b5-bffc-d6e17b3119f7 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,year,month,month_name,total_transaction,total_revenue
0,2024,1,January,4156,565325730.0
1,2024,2,February,3892,527792760.0
2,2024,3,March,4186,570216510.0
3,2024,4,April,4067,556243605.0
4,2024,5,May,4239,578362275.0
5,2024,6,June,4025,552664620.0
6,2024,7,July,4301,587718990.0
7,2024,8,August,4094,553053825.0
8,2024,9,September,3821,520073055.0
9,2024,10,October,3797,518623560.0


# <b>D. <span style='color:#0B2F9F'><code>SUB-QUERY</code></span></b>
Subquery adalah sebuah query (pernyataan SQL) yang terletak di dalam query lain. Subquery dapat digunakan untuk mengambil data yang kemudian akan digunakan oleh query utama.<br><br>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, hitung rata-rata umur customer dan bulatkan ke dua tempat desimal!</b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,age
0,34.04


**Latihan Query Mandiri :**

In [17]:
%%bigquery --project {project_id} --verbose

SELECT
  ROUND(AVG(age), 2) AS avg_age
FROM(
  SELECT*,
  DATE_DIFF(CURRENT_DATE(), PARSE_DATE('%Y-%m-%d', birth_date) ,YEAR) AS age
  FROM dqlab-9876543.dq_pizza.customers
);

Executing query with job ID: 2c9a620a-c52d-4c4c-97e6-e6b3cf4f6c93
Query executing: 0.54s
Job ID 2c9a620a-c52d-4c4c-97e6-e6b3cf4f6c93 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,avg_age
0,34.04


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customers, tampilkan customer_id, age dan gendernya dimana umur customer tersebut lebih dari rata-rata umur customer secara keseluruhan!</b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,customer_id,age,gender
0,CUST006458,42,F
1,CUST005844,39,F
2,CUST001212,44,F
3,CUST003654,35,F
4,CUST004438,43,F
...,...,...,...
64,CUST005867,37,M
65,CUST008255,45,M
66,CUST002681,47,M
67,CUST009522,36,M


**Latihan Query Mandiri :**

In [23]:
%%bigquery --project {project_id} --verbose

SELECT*
FROM(
  SELECT
  customer_id,
  DATE_DIFF(CURRENT_DATE(), PARSE_DATE('%Y-%m-%d', birth_date) ,YEAR) AS age,
  gender,
  FROM dqlab-9876543.dq_pizza.customers)
WHERE age >
  (SELECT
  AVG(DATE_DIFF(CURRENT_DATE(), PARSE_DATE('%Y-%m-%d', birth_date) ,YEAR)) AS avg_age,
  FROM dqlab-9876543.dq_pizza.customers
  )
ORDER BY age ASC
;

Executing query with job ID: 072724a5-fa9d-4761-97f4-f9e2c6e968ef
Query executing: 0.54s
Job ID 072724a5-fa9d-4761-97f4-f9e2c6e968ef successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,age,gender
0,CUST001309,35,F
1,CUST005429,35,M
2,CUST001128,35,M
3,CUST007150,35,F
4,CUST001597,35,F
...,...,...,...
64,CUST001137,49,M
65,CUST008584,50,F
66,CUST007809,50,M
67,CUST002835,51,M


# <b>E. <span style='color:#0B2F9F'><code>Common Table Expression (CTE)</code></span></b>
CTE, atau Common Table Expression, adalah fitur dalam SQL yang memungkinkan Anda untuk mendefinisikan sebuah query sementara yang dapat digunakan dalam query lain. CTE membantu dalam membuat query lebih mudah dibaca dan dipelihara, serta memungkinkan penggunaan rekursi dalam query.

<img src="https://learnsql.com/blog/sql-common-table-expression-guide/cte_syntax_example.webp" width="50%">

#### <i><b><span style='color:#55679C'>Quest</span> : Siapakah customer yang paling banyak spending money di DQPizza?</b></i>

**Contoh Hasil :**

In [ ]:
%%bigquery --project {project_id} --verbose

...

,customer_id,total_revenue
0,CUST006333,237372120.0


**Latihan Query Mandiri :**

In [30]:
%%bigquery --project {project_id} --verbose

WITH spendings AS (
  SELECT
    customer_id,
    SUM(price) AS total_revenue
  FROM dqlab-9876543.dq_pizza.f_detail_transaksi
  GROUP BY customer_id
  ORDER BY total_revenue DESC
  LIMIT 1
  )
SELECT *
FROM spendings;

Executing query with job ID: b6cfc3e4-654c-49ed-91e3-2cf7137fc232
Query executing: 0.61s
Job ID b6cfc3e4-654c-49ed-91e3-2cf7137fc232 successfully executed


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,total_revenue
0,CUST006333,232165440.0


Data Source : <i>https://mavenanalytics.io/challenges/maven-pizza-challenge</i> (dengan modifikasi)


---

<br>
<a href="https://www.linkedin.com/in/bachtiyarma/"><img src="https://img.shields.io/badge/-© 2025 Bachtiyar M. Arief-417DAC?style=for-the-badge&logoColor=white"/></a>

<a href="https://dqlab.id/"><img src="https://dqlab.id/files/dqlab/cache/87e30118ebba5ec7d96f6ea8c9dcc10b_x_118_X_55.png" align="left" /></a>
